# Задание
Используя стандартные инструменты переведите корпус htbg.txt в формат CoNLL-U. Используйте следующие колонки: 1. Номер токена в предложении 2. Токен в том виде, в котором он встретился в тексте 3. Лемма токена 4. POS-таг токена 5. Вектор грамматических значений токена 6. Целевая метка (сделайте метку везде OUT)

In [1]:
with open("htbg.txt", "r") as f:
    text = f.read()

In [2]:
from razdel import sentenize

sentences = [s.text for s in sentenize(text)]
sentences[:5]

['Аркадий и Борис Стругацкие\n Трудно быть богом\n\n\n «То были дни, когда я познал, что значит: страдать; что значит: стыдиться; что значит: отчаяться».',
 'Пьер Абеляр\n\n\n\n «Должен вас предупредить вот о чем.',
 'Выполняя задание, вы будете при оружии для поднятия авторитета.',
 'Но пускать его в ход вам не разрешается ни при каких обстоятельствах.',
 'Ни при каких обстоятельствах.']

In [3]:
from razdel import tokenize

tokens_by_sentence = []
for s in sentences:
    tokens_by_sentence.append([token.text for token in tokenize(s)])

tokens_by_sentence[0]

['Аркадий',
 'и',
 'Борис',
 'Стругацкие',
 'Трудно',
 'быть',
 'богом',
 '«',
 'То',
 'были',
 'дни',
 ',',
 'когда',
 'я',
 'познал',
 ',',
 'что',
 'значит',
 ':',
 'страдать',
 ';',
 'что',
 'значит',
 ':',
 'стыдиться',
 ';',
 'что',
 'значит',
 ':',
 'отчаяться',
 '»',
 '.']

In [4]:
from pymorphy2 import MorphAnalyzer
from russian_tagsets import converters

morph_analyzer = MorphAnalyzer()
to_ud = converters.converter('opencorpora-int', 'ud14')

token_properties_by_sentence = []
for tokens in tokens_by_sentence:
    token_properties_by_sentence.append([])
    for token in tokens:
        parse = morph_analyzer.parse(token)[0]
        ud_tag = to_ud(str(parse.tag))
        pos = ud_tag.split()[0]
        grammemes = ud_tag.split()[1]
        token_properties_by_sentence[-1].append((token, parse.normal_form, pos, grammemes))

token_properties_by_sentence[0]

[('Аркадий',
  'аркадий',
  'PROPN',
  'Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing'),
 ('и', 'и', 'CONJ', '_'),
 ('Борис', 'борис', 'PROPN', 'Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing'),
 ('Стругацкие', 'стругацкий', 'PROPN', 'Animacy=Anim|Case=Nom|Number=Ptan'),
 ('Трудно', 'трудно', 'ADV', '_'),
 ('быть', 'быть', 'VERB', 'Aspect=Imp|VerbForm=Inf'),
 ('богом', 'бог', 'NOUN', 'Animacy=Anim|Case=Ins|Gender=Masc|Number=Sing'),
 ('«', '«', 'PUNCT', '_'),
 ('То', 'то', 'CONJ', '_'),
 ('были',
  'быть',
  'VERB',
  'Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin'),
 ('дни', 'день', 'NOUN', 'Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur'),
 (',', ',', 'PUNCT', '_'),
 ('когда', 'когда', 'CONJ', '_'),
 ('я', 'я', 'PRON', 'Case=Nom|Number=Sing|Person=1'),
 ('познал',
  'познать',
  'VERB',
  'Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin'),
 (',', ',', 'PUNCT', '_'),
 ('что', 'что', 'CONJ', '_'),
 ('значит', 'значит', 'CONJ', '_'),
 (':', ':', 'PUNCT', '_'

In [5]:
with open('htbg.conllu', 'w') as f:
    for sentence, tokens in zip(sentences, token_properties_by_sentence):
        print("# text = {}".format(sentence), file=f)
        for i, token in enumerate(tokens):
            properties = [str(i + 1)]
            properties.extend(token)
            properties.append("OUT")
            print("\t".join(properties), file=f)
        print(file=f)